In [1]:
import z3

In [29]:
s = z3.String('s')
t = z3.String('t')
s, t

(s, t)

In [39]:
solver = z3.Solver()
solver.add(z3.PrefixOf('fo', s), z3.Length(s) > 5)
solver.check(), solver.model()

(sat, [s = "fo    "])

In [41]:
i = z3.Int('i')
z3.solve(i == z3.LastIndexOf('o', 'foobar'))

[i = -1]


Now to write rfind. This is the position of a substring where the rest of the string does not match any more.

In [93]:
s = 'this is foobared up!'
sub = 'foobar'
start = 1
end = 15

solver = z3.Solver()
index_remaining = z3.Int('result') + len(sub)
first_index = index_remaining - len(sub)
last_match = z3.SubString(s, index_remaining-len(sub), len(sub))
remaining = z3.SubString(s, index_remaining, len(s) - index_remaining)
solver.add(z3.And(z3.Contains(last_match, sub), z3.Not(z3.Contains(remaining, sub))))
solver.check(), solver.model()

(sat, [result = 8])

In [118]:
class Foo:
    def __init__(self):
        self.statespace = z3.Solver()
        self.value = 'this is foobared up!'
        
    def rfind(self, sub, start=None, end=None):
        start = 0 if start is None else start
        end = z3.Length(self.value) if end is None else end
        space = self.statespace
        value = self.value
        result = z3.Int('result')
        
        index_remaining = result + len(sub)
        last_match = z3.SubString(s, result, len(sub))
        remaining = z3.SubString(s, index_remaining, len(s) - index_remaining)
        found_match = z3.And(z3.Contains(last_match, sub), z3.Not(z3.Contains(remaining, sub)))
        no_match = z3.And(z3.Not(z3.Contains(value, sub)), result == -1)
        space.add(z3.Or(found_match, no_match))
        return result


In [119]:
f = Foo()
f.statespace.check(), f.statespace.model()

(sat, [])

In [120]:
f = Foo()
f.rfind('moobar')
f.statespace.check(), f.statespace.model()

(sat, [result = -1])

In [121]:
f = Foo()
f.rfind('foobar')
f.statespace.check(), f.statespace.model()

(sat, [result = 8])